In [1]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('FAU_Bank_Turnover.csv')

# Display the first 5 rows
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
print(df.info())

| job_satisfaction_level   | engagement_with_task   | last_performance_evaluation   | completed_projects   | average_working_hours_monthly   | years_spent_with_company   | received_support   | left   | promotion_last_5years   | job_role          | salary   |
|:-------------------------|:-----------------------|:------------------------------|:---------------------|:--------------------------------|:---------------------------|:-------------------|:-------|:------------------------|:------------------|:---------|
| 0.38                     | 0.19                   | 0.53                          | 2                    | 157                             | 3                          | 0                  | 1      | 0                       | investment_banker | low      |
| 0.8                      | 0.72                   | 0.86                          | 5                    | 262                             | 6                          | 0                  | 1      | 0                    

In [2]:
# Filter to only include employees who left
left_employees = df[df['left'] == 1]

In [3]:
# Calculate and print the mean job satisfaction level of employees who left
mean_job_satisfaction = left_employees['job_satisfaction_level'].mean()
print(f'Average job satisfaction level of employees who left: {mean_job_satisfaction:.2f}')

Average job satisfaction level of employees who left: 0.44


In [4]:
# Create a mapping of salary levels to numerical values
salary_mapping = {'low': 1, 'medium': 2, 'high': 3}

# Map the salary values in the filtered dataset to numerical values
left_employees['salary_mapped'] = left_employees['salary'].map(salary_mapping)

# Convert the `salary_mapped` column in `left_employees` to numeric datatype
left_employees['salary_mapped'] = pd.to_numeric(left_employees['salary_mapped'])

# Calculate and print the mean mapped salary of employees who left
mean_salary_mapped = left_employees['salary_mapped'].mean()
print(f'Average salary level of employees who left: {mean_salary_mapped:.2f}')

Average salary level of employees who left: 1.41


C:\Users\tanej\AppData\Local\Temp\ipykernel_45620\280759420.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left_employees['salary_mapped'] = left_employees['salary'].map(salary_mapping)
C:\Users\tanej\AppData\Local\Temp\ipykernel_45620\280759420.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left_employees['salary_mapped'] = pd.to_numeric(left_employees['salary_mapped'])


In [5]:
# Calculate and print the mean years spent at the company of employees who left
mean_years_spent = left_employees['years_spent_with_company'].mean()
print(f'Average years spent at the company for employees who left: {mean_years_spent:.2f}')

Average years spent at the company for employees who left: 3.88


In [6]:
# Calculate the number of employees who left for each salary level
left_counts_by_salary = left_employees['salary'].value_counts()

In [7]:
# Calculate the total number of employees who left
total_left = left_counts_by_salary.sum()
print(f'total number of employees who left: {total_left}')

total number of employees who left: 3571


In [8]:
# Calculate the proportion of employees who left for each salary level
left_proportions_by_salary = left_counts_by_salary / total_left

# Print the proportions of employees who left for each salary level
print("Proportion of employees who left for each salary level:")
print(left_proportions_by_salary.to_markdown(numalign="left", stralign="left"))

Proportion of employees who left for each salary level:
|        | salary    |
|:-------|:----------|
| low    | 0.608233  |
| medium | 0.368804  |
| high   | 0.0229628 |


In [12]:
import altair as alt
# Calculate the correlation matrix
correlation_matrix = df.corr()

# Print the correlation matrix
print("Correlation matrix:")
print(correlation_matrix.round(2).to_markdown(numalign="left", stralign="left"))

# Create a DataFrame from the correlation matrix
corr_df = correlation_matrix.reset_index().melt('index')

# Create the heatmap
heatmap = alt.Chart(corr_df).mark_rect().encode(
    x=alt.X('index:O', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('variable:O'),
    color=alt.Color('value:Q', scale=alt.Scale(scheme='blueorange')),
    tooltip=['index', 'variable', 'value']
).properties(
    title='Correlation Heatmap'
).interactive()

# Add text labels to the heatmap
text = heatmap.mark_text(baseline='middle').encode(
    text=alt.Text('value:Q', format='.2f'),
    color=alt.condition(
        alt.datum.value > 0.5, 
        alt.value('white'),
        alt.value('black')
    )
)

# Combine the heatmap and text layers
chart = heatmap + text

# Save the chart as a JSON file
chart.save('correlation_heatmap.json')
chart.show()

# Filter the correlation matrix to only include correlations with the 'left' column
correlations_with_left = correlation_matrix['left']

# Create a DataFrame from the filtered correlations
correlations_with_left_df = pd.DataFrame({'Feature': correlations_with_left.index, 'Correlation': correlations_with_left.values})

# Sort the DataFrame by absolute value of correlations in descending order
correlations_with_left_df['Abs_Correlation'] = correlations_with_left_df['Correlation'].abs()
correlations_with_left_df = correlations_with_left_df.sort_values(by='Abs_Correlation', ascending=False)

# Print the sorted correlations
print("Correlations with 'left':")
print(correlations_with_left_df.to_markdown(index=False, numalign="left", stralign="left"))

# Create a bar chart to visualize the correlations with the 'left' column
bar_chart = alt.Chart(correlations_with_left_df).mark_bar().encode(
    x=alt.X('Feature:N', sort='-y'),
    y=alt.Y('Correlation:Q'),
    tooltip=['Feature', 'Correlation']
).properties(
    title='Correlations with Employee Turnover'
).interactive()

# Save the chart as a JSON file
bar_chart.save('correlations_with_left_bar_chart.json')
bar_chart.show()

C:\Users\tanej\AppData\Local\Temp\ipykernel_45620\1301863367.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


Correlation matrix:
|                               | job_satisfaction_level   | engagement_with_task   | last_performance_evaluation   | completed_projects   | average_working_hours_monthly   | years_spent_with_company   | received_support   | left   | promotion_last_5years   |
|:------------------------------|:-------------------------|:-----------------------|:------------------------------|:---------------------|:--------------------------------|:---------------------------|:-------------------|:-------|:------------------------|
| job_satisfaction_level        | 1                        | -0.01                  | 0.11                          | -0.14                | -0.02                           | -0.1                       | 0.06               | -0.39  | 0.03                    |
| engagement_with_task          | -0.01                    | 1                      | -0                            | -0.01                | 0                               | -0                       

alt.LayerChart(...)

Correlations with 'left':
| Feature                       | Correlation   | Abs_Correlation   |
|:------------------------------|:--------------|:------------------|
| left                          | 1             | 1                 |
| job_satisfaction_level        | -0.388375     | 0.388375          |
| received_support              | -0.154622     | 0.154622          |
| years_spent_with_company      | 0.144822      | 0.144822          |
| average_working_hours_monthly | 0.0712872     | 0.0712872         |
| promotion_last_5years         | -0.0617881    | 0.0617881         |
| completed_projects            | 0.0237872     | 0.0237872         |
| engagement_with_task          | 0.0117488     | 0.0117488         |
| last_performance_evaluation   | 0.00656712    | 0.00656712        |


alt.Chart(...)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,log_loss

# Create a mapping of job roles to numerical values
job_role_mapping = {role: i for i, role in enumerate(df['job_role'].unique())}

# Map the job roles in the DataFrame to numerical values
df['job_role_mapped'] = df['job_role'].map(job_role_mapping)

# Convert the `salary` column to ordered categorical datatype
salary_order = ['low', 'medium', 'high']
df['salary'] = pd.Categorical(df['salary'], categories=salary_order, ordered=True)

# Bin the `job_satisfaction_level` into three equal-sized bins
df['job_satisfaction_binned'] = pd.qcut(df['job_satisfaction_level'], 3, labels=['low', 'medium', 'high'])

# Bin the `last_performance_evaluation` into three equal-sized bins
df['performance_binned'] = pd.qcut(df['last_performance_evaluation'], 3, labels=['low', 'medium', 'high'])

# Create a new feature called `workload`
df['workload'] = df['completed_projects'] * df['average_working_hours_monthly']

# Features (excluding the original columns that were transformed or are not relevant)
features = ['job_satisfaction_binned', 'performance_binned', 'workload', 'years_spent_with_company', 'received_support', 'promotion_last_5years']

In [14]:
# Separate features (X) and target variable (y)
X = pd.get_dummies(df[features], drop_first=True)  # One-hot encode categorical features
y = df['left']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to train and evaluate a model
def train_and_evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    return {
        'Model': model.__class__.__name__,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

# List of models to evaluate
models = [
    LogisticRegression(max_iter=1000),
    GradientBoostingClassifier(n_estimators=100, random_state=42),
    SVC(probability=True),  # Enable probability estimates for SVC
    MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
]

# Train and evaluate each model
results = []
for model in models:
    results.append(train_and_evaluate_model(model, X_train, X_test, y_train, y_test))

# Create a DataFrame of results
results_df = pd.DataFrame(results)

# Sort the results DataFrame by accuracy in descending order
results_df = results_df.sort_values(by='Accuracy', ascending=False)

# Print results
print("\nComparison of Model Performance:")
print(results_df.to_markdown(index=False, numalign="left", stralign="left"))


Comparison of Model Performance:
| Model                      | Accuracy   | Precision   | Recall   | F1 Score   |
|:---------------------------|:-----------|:------------|:---------|:-----------|
| GradientBoostingClassifier | 0.959      | 0.938346    | 0.883853 | 0.910284   |
| SVC                        | 0.873333   | 0.783972    | 0.637394 | 0.703125   |
| MLPClassifier              | 0.823333   | 0.585938    | 0.849858 | 0.693642   |
| LogisticRegression         | 0.731667   | 0.372751    | 0.205382 | 0.26484    |


In [15]:
# Create and train the Gradient Boosting Classifier model
model = GradientBoostingClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
logloss = log_loss(y_test, model.predict_proba(X_test)[:, 1])

# Print performance metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f'Log Loss: {logloss:.2f}')

Accuracy: 0.96
Precision: 0.94
Recall: 0.88
F1 Score: 0.91
Log Loss: 0.14


In [ ]:
# Get feature importances from the model
importances = model.feature_importances_

# Create a DataFrame of feature importances
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Sort by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print the sorted feature importances
print("\nFeature Importances:")
print(importance_df.to_markdown(index=False, numalign="left", stralign="left"))